# Exploring GPyTorch & KeOps for determinants

In [1]:
!pip install -q torch gpytorch pykeops

In [2]:
import torch
from gpytorch import lazy
torch.set_default_dtype(torch.double)

## Naive dense implementation

Let us begin by computing the partition function of a phylogeny-constrained spanning tree distribution.

In [3]:
n = 3000  # number of leaves
d = 9  # embedding dimension

V = torch.arange(n)
U = torch.arange(n, 2 * n - 1)

torch.manual_seed(0)
t_V = torch.randn(n)
t_U = torch.randn(n - 1) - 1
t = torch.cat([t_V, t_U])

z = torch.randn(2 * n - 1, d)

In [4]:
dt = t[:, None] - t[None, :]
abs_dt = dt.abs().clamp(min=1e-2)  # avoid nans
dz = z[:, None] - z[None, :]
w = dz.pow(2).sum(-1).div(abs_dt).mul(-0.5).exp() / abs_dt.pow(d / 2)

# Exclude leaf-leaf edges.
is_leaf = torch.cat([torch.ones(n), torch.zeros(n - 1)])
w = w * (1 - is_leaf * is_leaf[:, None])

# Exclude internal-leaf edges that are out of order.
ooo = is_leaf[:, None] * (1 - is_leaf) * (dt <= 0).float()
w = w * (1 - ooo - ooo.transpose(-1, -2))

L = w.sum(dim=-1).diag_embed() - w
L = L + torch.eye(2 * n - 1) * 1e-8  # avoid nans

In [5]:
%%time
L.logdet()

CPU times: user 3.61 s, sys: 133 ms, total: 3.74 s
Wall time: 3.77 s


tensor(14711.0272)

## Speed up using GPyTorch's approximation

In [6]:
L_ = lazy.NonLazyTensor(L)

In [7]:
%%time
L_.logdet()

CPU times: user 1.22 s, sys: 24 ms, total: 1.24 s
Wall time: 1.25 s


tensor(15075.6427)

## Reduce memory using KeOps

In [8]:
from pykeops.torch import LazyTensor

if False:  # TODO
    t_ = LazyTensor(t)
    z_ = LazyTensor(z)
    
    dt = t_ - t_.transpose(-1, -2)
    abs_dt = dt.abs().clamp(min=1e-4)